In [1]:
library(caret) #sample
library(nnet) #인공신경망 , 다항 로지스틱 회귀분석
library(e1071) # svm
library(randomForest) # randomForest
library(party) # 의사결정나무 ctree

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich


In [2]:
# 표준화
iris2 <- iris
iris2[,1:4] <- as.data.frame(scale(iris[,1:4]))

In [3]:
# 샘플나누기

sampling <- function(){
samp <<- createDataPartition(iris2$Species , p = 0.7,list = F) #<<-를 하는 이유는 외부데이터를 손상되게 하지 않기 위함이다.
data.tr <<- iris2[samp,]                                       #단, function내에서만 가능하다.
data.test <<- iris2[-samp,]

x <<- subset(data.test, select = -Species)
y <<- data.test$Species
    
}    


In [4]:
sampling() 
head(samp)

Resample1
1
4
6
8
9
11


In [5]:
#인공신경망 테스트
ANN <- function(data.tr , data.test , x , y ){
    model.nnet <- nnet(Species ~. , data = data.tr , size = 3)
    pred <- predict(model.nnet , x , type = "class")
    cat("예측력" ,mean(pred == y), "입니다.")
    table(pred,y)
    return(mean(pred == y))
}

In [6]:
ANN(data.tr , data.test , x , y )

# weights:  27
initial  value 118.942486 
iter  10 value 9.376956
iter  20 value 5.421462
iter  30 value 4.848747
iter  40 value 2.849455
iter  50 value 2.582820
iter  60 value 2.514409
iter  70 value 2.512740
iter  80 value 2.512677
iter  90 value 2.504322
iter 100 value 2.499741
final  value 2.499741 
stopped after 100 iterations
예측력 0.9777778 입니다.

[1] 0.9777778

In [7]:
#의사결정 나무 
CTREE <-function(data.tr , data.test , x , y ){
    model.ctree <- ctree(Species ~ . , data=data.tr)
    pred <- predict(model.ctree , x )
    cat("예측력" ,mean(pred == y), "입니다.")
    table(pred,y)
    return(mean(pred == y))
}

In [8]:
 CTREE(data.tr , data.test , x , y )

예측력 0.9777778 입니다.

[1] 0.9777778

In [9]:
#RandomForesst
RandomForest<-function(data.tr , data.test , x , y ){
    model.rf <- randomForest(Species ~ . , data=data.tr, ntree = 100)
    pred <- predict(model.rf , x )
    cat("예측력" ,mean(pred == y), "입니다.")
    table(pred,y)
    return(mean(pred == y))
} 

In [10]:
#SVM
SVM <-function(data.tr , data.test , x , y ){
    model.svm <- svm(Species ~ . , data=data.tr)
    pred <- predict(model.svm , x )
    cat("예측력" ,mean(pred == y), "입니다.")
    table(pred,y)
    return(mean(pred == y))
} 

In [11]:
#Logistic regression
Logisticreg <-function(data.tr , data.test , x , y ){
    model.log <- multinom(Species ~ . , data=data.tr)
    pred <- predict(model.log , x )
    cat("예측력" ,mean(pred == y), "입니다.")
    table(pred,y)
    return(mean(pred == y))
} 

In [12]:
a <- rep(0,5)
iteration <- 5
for(i in 1:iteration){
    
    a = a + c(
            ANN(data.tr , data.test , x , y ),
            CTREE(data.tr , data.test , x , y ),
            RandomForest(data.tr , data.test , x , y ),
            SVM(data.tr , data.test , x , y ),
            Logisticreg(data.tr , data.test , x , y )
    )
    
}
a<- a/iteration

result<-data.frame(예측력 = a)
rownames(result) = c("인공신경망","의사결정나무","RandomForest","SVM","로지스틱회귀분석")
result

# weights:  27
initial  value 114.293171 
iter  10 value 10.011981
iter  20 value 5.049550
iter  30 value 4.769650
iter  40 value 4.669557
iter  50 value 4.523197
iter  60 value 3.542283
iter  70 value 3.285796
iter  80 value 2.534845
iter  90 value 2.003490
iter 100 value 1.975240
final  value 1.975240 
stopped after 100 iterations
예측력 0.9555556 입니다.예측력 0.9777778 입니다.예측력 0.9777778 입니다.예측력 1 입니다.# weights:  18 (10 variable)
initial  value 115.354290 
iter  10 value 13.176781
iter  20 value 5.323601
iter  30 value 5.308446
iter  40 value 5.303851
iter  50 value 5.300993
iter  60 value 5.300580
iter  70 value 5.299575
iter  80 value 5.299314
iter  90 value 5.299060
iter 100 value 5.298901
final  value 5.298901 
stopped after 100 iterations
예측력 1 입니다.# weights:  27
initial  value 143.986036 
iter  10 value 23.690514
iter  20 value 6.548084
iter  30 value 5.497091
iter  40 value 2.264104
iter  50 value 0.153754
iter  60 value 0.008709
iter  70 value 0.001569
iter  80 value 0.000238
iter  9

,예측력
인공신경망,0.9777778
의사결정나무,0.9777778
RandomForest,0.9688889
SVM,1.0000000
로지스틱회귀분석,1.0000000


In [14]:
result

,예측력
인공신경망,0.9777778
의사결정나무,0.9777778
RandomForest,0.9688889
SVM,1.0000000
로지스틱회귀분석,1.0000000
